In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,7651
2,Huelva,Confirmados PDIA 14 días,2566
3,Huelva,Tasa PDIA 14 días,"502,40531930931996"
4,Huelva,Confirmados PDIA 7 días,1408
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,7
629,Municipio de Huelva sin especificar,Total Confirmados,107
630,Municipio de Huelva sin especificar,Curados,26


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  7651.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2093.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 375 personas en los últimos 7 días 

Un positivo PCR cada 194 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,7651.0,1408.0,2566.0,510743.0,275.676808,502.405319,564.0
Huelva-Costa,4042.0,817.0,1484.0,288115.0,283.567326,515.072107,345.0
Condado-Campiña,2391.0,414.0,717.0,155057.0,266.998588,462.410597,170.0
Huelva (capital),2093.0,383.0,740.0,143663.0,266.596131,515.094353,162.0
Lepe,626.0,137.0,230.0,27431.0,499.434946,838.467427,48.0
Sierra de Huelva-Andévalo Central,1112.0,170.0,351.0,67571.0,251.587219,519.453612,44.0
Cartaya,230.0,69.0,114.0,19974.0,345.449084,570.741965,39.0
Palma del Condado (La),351.0,69.0,108.0,10761.0,641.204349,1003.624198,35.0
Ayamonte,187.0,45.0,76.0,20946.0,214.838155,362.837773,27.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Aracena,273.0,38.0,120.0,8107.0,468.730727,1480.202294,4.0
Villablanca,49.0,22.0,36.0,2848.0,772.471910,1264.044944,10.0
Almendro (El),16.0,8.0,10.0,826.0,968.523002,1210.653753,1.0
Palma del Condado (La),351.0,69.0,108.0,10761.0,641.204349,1003.624198,35.0
Villarrasa,85.0,16.0,19.0,2176.0,735.294118,873.161765,4.0
Campillo (El),47.0,8.0,17.0,2023.0,395.452299,840.336134,3.0
Lepe,626.0,137.0,230.0,27431.0,499.434946,838.467427,48.0
Marines (Los),5.0,3.0,3.0,396.0,757.575758,757.575758,1.0
Valverde del Camino,245.0,55.0,94.0,12820.0,429.017161,733.229329,12.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Calañas,35.0,1.0,5.0,2791.0,35.829452,179.147259,1.0,0.200000
San Silvestre de Guzmán,7.0,1.0,4.0,623.0,160.513644,642.054575,1.0,0.250000
Encinasola,7.0,1.0,4.0,1305.0,76.628352,306.513410,0.0,0.250000
Gibraleón,315.0,24.0,92.0,12607.0,190.370429,729.753312,7.0,0.260870
Cortegana,124.0,2.0,7.0,4666.0,42.863266,150.021432,0.0,0.285714
Manzanilla,44.0,3.0,10.0,2135.0,140.515222,468.384075,1.0,0.300000
Aracena,273.0,38.0,120.0,8107.0,468.730727,1480.202294,4.0,0.316667
Aroche,36.0,3.0,9.0,3073.0,97.624471,292.873414,1.0,0.333333
Zarza-Perrunal (La),12.0,2.0,6.0,1267.0,157.853197,473.559590,1.0,0.333333
